In [28]:
# IMPORTS

#---------------------------------------
import numpy as np

In [29]:
# COORDINATES FUNCTIONS

#----------------------------------------------------------------
# Returns the 3D vector (x, y, z) correspoding to the spherical_coordinates (r, theta, phi)
# Imput parameter is strictly a numpy array of 3 numbers: (r, theta, phi)
def spherical_to_cartesian(spherical_coordinates):
    r = spherical_coordinates[0]
    theta = spherical_coordinates[1]
    phi = spherical_coordinates[2]

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    # Floating point precision would not give exact results around zeros of goniometric functions
    return np.array([x, y, z])

In [30]:
# DISTRIBUTION GENERATION FUNCTIONS - (HOMOGENEOUS SPHERE)

#---------------------------------------------------------------
# Generate a random radius within "sphere_radius" by sampling from its PDF
# This is achieved by using its inverse CDF
def generate_radius(sphere_radius, size=1):
    P = np.random.uniform(0, 1, size)

    return sphere_radius * (P ** (1 / 3))

# Generate a random inclination theta by sampling from its PDF
# This is achieved by using its inverse CDF
def generate_theta(size=1):
    P = np.random.uniform(0, 1, size)

    return np.arccos(1 - 2 * P)

# Generate a random azimuth phi; phi is uniformly distributed
def generate_phi(size=1):
    return np.random.uniform(0, 2 * np.pi, size)

# Generate a random set of coordinates of a point within "sphere_radius"
# The returned value is shaped like this: [[r_0, theta_0, phi_0],
#                                         [r_1, theta_1, phi_1],
#                                         ...
#                                         [r_(size-1), theta_(size-1), phi_(size-1)]]
def generate_sphere_point(sphere_radius, size=1):
    r = generate_radius(sphere_radius, size)
    theta = generate_theta(size)
    phi = generate_phi(size)

    # np.stack() merges the coordinates into the desired shape
    return np.stack((r, theta, phi), axis=1)

In [31]:
# PHYSICS FUNCTIONS

#-------------------------------------------------
def get_sphere_volume(radius):
    return (4 * np.pi / 3) * radius ** 3

def get_total_mass(part_mass, part_N):
    return part_mass * part_N

def get_average_density(mass, volume):
    return mass / volume

def get_collapse_time(density): # G = 1
    T_w = (3 * np.pi / density) ** (1 / 2)
    return T_w / (4 * 2 ** (1 / 2))

# Radius of a sphere of volume = sphere_volume / N
def get_mean_separation(number_density):
    return ((3 / (4 * np.pi)) / number_density) ** (1 / 3)

def speed_at_radius(radius, radius_0, total_mass): # G = 1
    return (2 * total_mass * (1 / radius - 1 / radius_0)) ** (1 / 2)

In [32]:
# PARAMETERS OF THE DISTRIBUTION

N = 30000 # Number of points
R = 10 # Radius of the sphere
t0 = 0 # Initial time. To be writed in the input_file
m = 1 # Mass of every any particle

In [33]:
# Generate the distribution
points_spherical = generate_sphere_point(R, N)

# Compute the cartesian coordinates of every particle and store it into an array
points_cartesian = np.array([spherical_to_cartesian(points_spherical[i]) for i in range(N)])

In [34]:
sphere_volume = get_sphere_volume(R)
total_mass = get_total_mass(m, N)
number_density = N / sphere_volume
average_density = total_mass / sphere_volume
number_density = N / sphere_volume
collapse_time = get_collapse_time(average_density)

mean_separation = get_mean_separation(N / get_sphere_volume(R))

radius_fraction = 1 / 100
collapse_radius = R * radius_fraction

mean_separation_close_to_collapse = get_mean_separation(N / get_sphere_volume(collapse_radius))
mean_speed_close_to_collapse = speed_at_radius(collapse_radius, R, total_mass)

suggested_time_step = collapse_radius / mean_speed_close_to_collapse

print("Collapse time: " + str(collapse_time))
print("Mean velocity at collapse radius R * " + str(radius_fraction) + ": " + str(mean_speed_close_to_collapse))
print("Suggested time step: " + str(suggested_time_step))
print("Mean separation at collapse radius R: " + str(mean_separation))
print("Mean separation at collapse radius R * " + str(radius_fraction) + ": " + str(mean_separation_close_to_collapse))

Collapse time: 0.20278893379868057
Mean velocity at collapse radius R * 0.01: 770.7139547199077
Suggested time step: 0.0001297498240269205
Mean separation at collapse radius R: 0.3218297948685433
Mean separation at collapse radius R * 0.01: 0.0032182979486854338


In [35]:
# Every particle is generated at rest
velocity_coord_str = "0 0 0"

masses = []
positions = []
velocities = []

for i in range(N):
    masses.append('\n' + str(m))
    positions.append('\n' + ' '.join(str(coord) for coord in points_cartesian[i]))
    velocities.append('\n' + velocity_coord_str)

In [36]:
make_input = False

if make_input:
    # Create or overwrite the input file
    file_name = "input.txt"

    input_file = open(file_name, 'w')

    # Write a file in the proper format for the treecode program:
    #
    # N
    # n_dim
    # t_0
    # m_1
    # ..
    # m_N
    # x_1, y_1, z_1
    # ...
    # x_N, y_N, z_N
    # vx_1, vy_1, vz_1
    # ...
    # vx_N, vy_N, vz_N
    input_file.write(str(N))
    input_file.write('\n' + str(3)) # Number of dimensions
    input_file.write('\n' + str(t0))
    input_file.writelines(masses)
    input_file.writelines(positions)
    input_file.writelines(velocities)

    input_file.close()